In [1]:
import napari
import numpy as np
from pathlib import Path
from tifffile import imread

from napari_cellseg3d.dev_scripts import evaluate_labels as eval
from napari_cellseg3d.utils import resize
from napari_cellseg3d.code_models.model_instance_seg import (
    binary_connected,
    binary_watershed,
    voronoi_otsu,
    to_semantic,
)

In [2]:
viewer = napari.Viewer()

In [3]:
im_path = Path("C:/Users/Cyril/Desktop/test/instance_test")
prediction_path = str(im_path / "pred.tif")
gt_labels_path = str(im_path / "labels_relabel_unique.tif")

prediction = imread(prediction_path)
gt_labels = imread(gt_labels_path)

zoom = (1 / 5, 1, 1)
prediction_resized = resize(prediction, zoom)
gt_labels_resized = resize(gt_labels, zoom)


viewer.add_image(prediction_resized, name="pred", colormap="inferno")
viewer.add_labels(gt_labels_resized, name="gt")

<Labels layer 'gt' at 0x2350d649130>

In [4]:
from napari_cellseg3d.utils import dice_coeff

dice_coeff(to_semantic(gt_labels_resized.copy()), to_semantic(prediction_resized.copy()))

0.5817600487210719

In [5]:
from napari_cellseg3d.dev_scripts.correct_labels import relabel

# gt_corrected = relabel(prediction_path, gt_labels_path, go_fast=False)

In [6]:
# eval.evaluate_model_performance(gt_labels_resized, gt_labels_resized)

In [7]:
print(prediction_resized.shape)
print(gt_labels_resized.shape)
print(np.unique(gt_labels_resized).shape[0])

(25, 64, 64)
(25, 64, 64)
2


In [8]:
connected = binary_connected(prediction_resized,thres_small=2)
viewer.add_labels(connected, name="connected")

<Labels layer 'connected' at 0x235263cbb50>

In [9]:
eval.evaluate_model_performance(gt_labels_resized, connected)

2023-03-22 15:48:05,891 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4352.70it/s]

2023-03-22 15:48:05,919 - Calculating the number of neurons not found...
2023-03-22 15:48:05,921 - Percent of non-fused neurons found: 0.00%
2023-03-22 15:48:05,922 - Percent of fused neurons found: 0.00%
2023-03-22 15:48:05,922 - Overall percent of neurons found: 0.00%


(0, 0, 1, 12, nan, nan, nan, nan, 1.0)

In [10]:
watershed = binary_watershed(
    prediction_resized, thres_small=2, rem_seed_thres=1
)
viewer.add_labels(watershed)
eval.evaluate_model_performance(gt_labels_resized, watershed)

2023-03-22 15:48:05,995 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4354.19it/s]

2023-03-22 15:48:06,021 - Calculating the number of neurons not found...
2023-03-22 15:48:06,023 - Percent of non-fused neurons found: 0.00%
2023-03-22 15:48:06,023 - Percent of fused neurons found: 0.00%
2023-03-22 15:48:06,024 - Overall percent of neurons found: 0.00%


(0, 0, 1, 10, nan, nan, nan, nan, 1.0)

In [11]:
voronoi = voronoi_otsu(prediction_resized, 1, outline_sigma=1)

from skimage.morphology import remove_small_objects

voronoi = remove_small_objects(voronoi, 2)
viewer.add_labels(voronoi)
voronoi.shape

(25, 64, 64)

In [12]:
gt_labels_resized.dtype

dtype('int64')

In [13]:
# np.unique(voronoi, return_counts=True)

In [14]:
# np.unique(gt_labels_resized, return_counts=True)

In [15]:
eval.evaluate_model_performance(gt_labels_resized, voronoi)

2023-03-22 15:48:06,360 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 4153.73it/s]

2023-03-22 15:48:06,392 - Calculating the number of neurons not found...
2023-03-22 15:48:06,394 - Percent of non-fused neurons found: 0.00%
2023-03-22 15:48:06,395 - Percent of fused neurons found: 0.00%
2023-03-22 15:48:06,395 - Overall percent of neurons found: 0.00%


(0, 0, 1, 17, nan, nan, nan, nan, 0.7306099091287442)

In [16]:
# eval.evaluate_model_performance(gt_labels_resized, voronoi)